In [1]:
alpha_name        = 'Buy_and_Hold_Resize_Daily'

desired_direction = 'long'  

how_many_bars     = 1

start_date        = '2020-12-31'#'2019-12-31'
end_date          = '2026-01-01'#'2025-07-01'

symbol_list = ['BTC']#, 'ETH', 'ADA', 'BNB', 'DOGE', 'LINK', 'XRP']#, 'AVAX', 'NEAR', 'SOL']

In [2]:
from twsq.alpha import Alpha

In [3]:
class Constants():
    
    def __init__(self):        
        self.exchange           = 'Binance'  #'Binance' or 'Kraken'       
        self.order_type         = 'limit'
        self.gross_trade_amount = 10000     # USD$10,000
        self.denominator        = 'USDT'
        self.rebal_frequency    = '1d'
                               
constants = Constants()

In [4]:
constants.alpha_name         = alpha_name
constants.desired_direction  = desired_direction
constants.how_many_bars      = how_many_bars
constants.start_date         = start_date
constants.end_date           = end_date 

In [5]:
class Buy_and_Hold_Resize_Daily(Alpha):
    
    def prepare(self):                        
        self.asset_pair    = constants.symbol + "/" + constants.denominator
            
    def rebalance(self):        
        self.cancel_all_orders()
        
        current_positions_dict = self.get_pos()    
        for asset in [constants.symbol, constants.denominator]:
            if asset not in current_positions_dict.keys():
                current_positions_dict[asset] = 0
                
        current_direction = ('long'  if current_positions_dict[constants.symbol] > 0 else
                             'short' if current_positions_dict[constants.symbol] < 0 else
                             'neutral')
                
        bars = self.get_lastn_bars(self.asset_pair, 
                                   constants.how_many_bars,
                                   constants.rebal_frequency)
        
        current_prices_dict = {constants.symbol      : bars["close"].iloc[-1], 
                               constants.denominator : 1}   
        
##########

        symbol_dollar_value = current_positions_dict[constants.symbol] * current_prices_dict[constants.symbol]
        symbol_dollar_value_to_trade = abs(constants.gross_trade_amount - symbol_dollar_value)
        order_amt = symbol_dollar_value_to_trade / current_prices_dict[constants.symbol]

##########        

        if order_amt == 0:
            return
        
        buy_sell = ('buy'  if symbol_dollar_value < constants.gross_trade_amount else     
                    'sell' if symbol_dollar_value > constants.gross_trade_amount else
                    'error')
        
        self.create_order(self.asset_pair,
                          order_amt,   
                          buy_sell,    
                          current_prices_dict[constants.symbol],
                          route=True)    

In [6]:
def run_result():

    directory_name = constants.alpha_name + "/" + \
                     constants.start_date + " " + \
                     constants.end_date   + " " + \
                     constants.symbol     
        
    print('\n')
    print(directory_name)
  
    result = Buy_and_Hold_Resize_Daily.run_backtest(start_ts = constants.start_date, 
                                                    end_ts   = constants.end_date, 
                                                    freq     = constants.rebal_frequency,
                                                    name     = directory_name)    

In [7]:
for symbol in symbol_list:        
    constants.symbol = symbol
    run_result()
        
print('\n')
print('done')



Buy_and_Hold_Resize_Daily/2020-12-31 2026-01-01 BTC
Running Buy_and_Hold_Resize_Daily/2020-12-31 2026-01-01 BTC backtest: 100.00% done | Total PnL (USDT): 19,146 | Duration (s): 17  

done
